# Homework Session 11 Neural networks

The purpose of the below is to classify days over years 2017-2018 by their corresponding mobility patterns between 10 zones in Taipei (quantified by an aggregated temporal network of subway ridership flows across the city)

In [10]:
import numpy as np
import pandas as pd
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import heapq

In [23]:
#read the data
TNet=pd.read_csv('taipeiD_TNet2.csv',header=None);

In [24]:
TNet.head() 
#each row represents a 10x10 adjacency matrix of the normalized Taipei subway mobility network between 10 zones flattened into a 100x1 row corresponding to a single day
#days start at jan-1-2017

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.017943,0.005415,0.003590,0.008316,0.007859,0.012942,0.012196,0.019543,0.001196,0.003327,...,0.002529,0.001533,0.001860,0.002375,0.005408,0.008922,0.003945,0.011075,0.005073,0.012708
1,0.021283,0.005215,0.003530,0.009359,0.007803,0.014288,0.011185,0.019044,0.001382,0.003499,...,0.002803,0.001757,0.001783,0.002549,0.005515,0.009650,0.003596,0.009618,0.005946,0.013709
2,0.028988,0.006511,0.005591,0.012970,0.007816,0.015878,0.010973,0.015768,0.002252,0.005388,...,0.004649,0.002555,0.002672,0.004291,0.007385,0.009558,0.004293,0.008791,0.010040,0.016301
3,0.029534,0.006471,0.005615,0.013017,0.007717,0.016098,0.011182,0.015815,0.002325,0.005443,...,0.004611,0.002473,0.002636,0.004195,0.007255,0.009487,0.004316,0.008729,0.010296,0.016437
4,0.029333,0.006525,0.005727,0.013098,0.007692,0.016358,0.011000,0.015677,0.002344,0.005527,...,0.004694,0.002515,0.002677,0.004222,0.007269,0.009921,0.004387,0.008923,0.010381,0.016914


In [103]:
#convert to an array and scale the data
X=np.array(TNet);

In [104]:
X=MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(X)

In [105]:
X.shape

(669, 100)

In [133]:
#define day of the week corresponding to each day of observation; 0-Sunday, 1-Monday,...,6-Saturday
y=np.array(range(669))%7; y[:10]

array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2])

In [134]:
yc=np_utils.to_categorical(y) #get categorical binary variables isSunday, isMonday,...
yc[:5]

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [135]:
X_test=X[400:,:]; X_train=X[:400,:]; #split the data into training and test
y_test=yc[400:,:]; y_train=yc[:400,:]

## Task 1. Classify weekdays/weekends
Label the rows with ones for weekends, zeros for weekdays.
Train a neural network with 4 layers of 30,10,3 and 1 (output) neurons over the training sample against this label, evaluating its performance over the test sample. Report the acheived accuracy (categorical) over the test sample

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 100 epochs, batch_size=20. 

In [128]:
np.random.seed(2019)
model = Sequential()
model.add(Dense(30, activation='relu', input_dim=100))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train[:,0]+y_train[:,6]>0, validation_data=(X_test, y_test[:,0]+y_test[:,6]>0), epochs=100, batch_size=20, verbose=2)
y_pred = model.predict(X_test)

Train on 400 samples, validate on 269 samples
Epoch 1/100
 - 4s - loss: 0.4961 - acc: 0.8775 - val_loss: 0.3209 - val_acc: 0.9442
Epoch 2/100
 - 0s - loss: 0.2525 - acc: 0.9550 - val_loss: 0.1911 - val_acc: 0.9554
Epoch 3/100
 - 0s - loss: 0.1736 - acc: 0.9525 - val_loss: 0.1479 - val_acc: 0.9554
Epoch 4/100
 - 0s - loss: 0.1453 - acc: 0.9525 - val_loss: 0.1331 - val_acc: 0.9554
Epoch 5/100
 - 0s - loss: 0.1337 - acc: 0.9525 - val_loss: 0.1258 - val_acc: 0.9554
Epoch 6/100
 - 0s - loss: 0.1274 - acc: 0.9550 - val_loss: 0.1209 - val_acc: 0.9554
Epoch 7/100
 - 0s - loss: 0.1213 - acc: 0.9550 - val_loss: 0.1159 - val_acc: 0.9554
Epoch 8/100
 - 0s - loss: 0.1160 - acc: 0.9550 - val_loss: 0.1119 - val_acc: 0.9591
Epoch 9/100
 - 0s - loss: 0.1129 - acc: 0.9550 - val_loss: 0.1107 - val_acc: 0.9591
Epoch 10/100
 - 0s - loss: 0.1075 - acc: 0.9550 - val_loss: 0.1049 - val_acc: 0.9591
Epoch 11/100
 - 0s - loss: 0.1036 - acc: 0.9550 - val_loss: 0.1072 - val_acc: 0.9554
Epoch 12/100
 - 0s - loss: 0

In [127]:
#Test accuracy
1.0*sum((y_pred.flatten()>0.5)==(y_test[:,0]+y_test[:,6]>0))/y_test.shape[0]

0.9702602230483272

## Task 2. Classify all days of the week
Train a neural network against the origial categorical label. Use 5 layers of 40,15,5 and 7 (outputs, representing probabilities for a current input to correspond to each of the weekdays) neurons over the training sample, evaluating its performance over the test sample (use 'categorical_accurary'). Report the acheived accuracy (categorical) over the test sample.

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 200 epochs, batch_size=20

In [129]:
np.random.seed(2019)
model = Sequential()
model.add(Dense(40, activation='relu', input_dim=100))
model.add(Dense(15, activation='relu', input_dim=40))
model.add(Dense(5, activation='relu', input_dim=7))
model.add(Dense(7, activation='sigmoid', input_dim=7))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=20, verbose=2)
y_pred = model.predict(X_test)

Train on 400 samples, validate on 269 samples
Epoch 1/200
 - 4s - loss: 0.6796 - categorical_accuracy: 0.1900 - val_loss: 0.6571 - val_categorical_accuracy: 0.1301
Epoch 2/200
 - 0s - loss: 0.6340 - categorical_accuracy: 0.1700 - val_loss: 0.6197 - val_categorical_accuracy: 0.1450
Epoch 3/200
 - 0s - loss: 0.6110 - categorical_accuracy: 0.1425 - val_loss: 0.6029 - val_categorical_accuracy: 0.1450
Epoch 4/200
 - 0s - loss: 0.5949 - categorical_accuracy: 0.1425 - val_loss: 0.5883 - val_categorical_accuracy: 0.1450
Epoch 5/200
 - 0s - loss: 0.5808 - categorical_accuracy: 0.1425 - val_loss: 0.5746 - val_categorical_accuracy: 0.1450
Epoch 6/200
 - 0s - loss: 0.5676 - categorical_accuracy: 0.1425 - val_loss: 0.5612 - val_categorical_accuracy: 0.1450
Epoch 7/200
 - 0s - loss: 0.5540 - categorical_accuracy: 0.1450 - val_loss: 0.5483 - val_categorical_accuracy: 0.1487
Epoch 8/200
 - 0s - loss: 0.5408 - categorical_accuracy: 0.1650 - val_loss: 0.5355 - val_categorical_accuracy: 0.1673
Epoch 9/20

In [130]:
#Output label
np.argmax(y_pred,axis=1)

array([3, 2, 4, 4, 5, 6, 0, 4, 4, 5, 1, 6, 1, 1, 1, 1, 2, 2, 5, 6, 0, 3,
       5, 6, 3, 5, 6, 0, 3, 3, 3, 3, 5, 6, 0, 3, 3, 3, 4, 5, 6, 0, 2, 3,
       3, 4, 5, 6, 0, 2, 3, 3, 4, 5, 6, 0, 4, 5, 6, 0, 6, 0, 1, 2, 3, 3,
       4, 5, 6, 0, 2, 3, 3, 3, 5, 6, 0, 2, 3, 3, 3, 5, 6, 0, 2, 3, 3, 3,
       5, 6, 0, 2, 3, 3, 3, 5, 6, 0, 2, 3, 3, 3, 5, 6, 0, 2, 3, 3, 3, 5,
       6, 0, 2, 3, 3, 3, 5, 6, 0, 2, 3, 3, 4, 5, 6, 0, 3, 2, 3, 3, 5, 6,
       0, 1, 2, 3, 3, 5, 6, 0, 2, 2, 2, 3, 5, 6, 0, 2, 2, 2, 4, 5, 6, 0,
       2, 2, 2, 3, 5, 6, 0, 2, 3, 2, 3, 5, 6, 0, 2, 2, 3, 4, 5, 6, 0, 2,
       2, 2, 4, 5, 6, 0, 2, 2, 2, 2, 5, 6, 0, 2, 2, 2, 3, 5, 6, 0, 2, 2,
       2, 2, 5, 6, 0, 2, 2, 2, 3, 5, 6, 0, 2, 2, 2, 3, 5, 6, 0, 3, 3, 3,
       3, 5, 6, 0, 2, 3, 3, 4, 5, 6, 0, 1, 2, 3, 3, 5, 6, 0, 2, 3, 3, 3,
       5, 6, 0, 2, 4, 0, 3, 5, 6, 0, 2, 3, 3, 4, 5, 6, 0, 2, 3, 3, 3, 5,
       6, 0, 2, 3, 3])

In [131]:
#Test accuracy
1.0*sum(y[400:]==np.argmax(y_pred,axis=1))/len(y[400:])

0.5910780669144982

## Task 3

Try the facial recognition task from NN lab 2 with different model specifications and report model accuracy:
1. Try different dropouts - 0.2, 0.3, 0.4
2. Try different convolution windows: (3$\times$3, 10$\times$10)

In [5]:
# load the data

from sklearn.datasets import fetch_lfw_people 
from keras.datasets import mnist

lfw_people = fetch_lfw_people(min_faces_per_person = 70, resize = 0.4) 
  
# the images arrays to find the shapes (for plotting) 
n_samples, h, w = lfw_people.images.shape 
  
# Instead of providing 2D data, X has data already in the form  of a vector that 
# is required in this approach. 
X = lfw_people.data 
n_features = X.shape[1] 

y = lfw_people.target 
target_names = lfw_people.target_names 
n_classes = target_names.shape[0] 

In [6]:
X_train, X_test, y_train, y_test = train_test_split( 
    X, y, test_size = 0.2)

X_train_pp = (X_train.reshape(X_train.shape[0], 50, 37, 1).astype('float32')) / 255 # normalize
X_test_pp = (X_test.reshape(X_test.shape[0], 50, 37, 1).astype('float32')) / 255

y_train_pp = np_utils.to_categorical(y_train)
y_test_pp = np_utils.to_categorical(y_test)

num_classes = y_train_pp.shape[1]

In [7]:
# model

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 1s - loss: 1.8786 - accuracy: 0.2738 - val_loss: 1.7614 - val_accuracy: 0.4186
Epoch 2/50
6/6 - 1s - loss: 1.6880 - accuracy: 0.4097 - val_loss: 1.6804 - val_accuracy: 0.4186
Epoch 3/50
6/6 - 1s - loss: 1.6674 - accuracy: 0.4097 - val_loss: 1.6774 - val_accuracy: 0.4186
Epoch 4/50
6/6 - 1s - loss: 1.6476 - accuracy: 0.4097 - val_loss: 1.6599 - val_accuracy: 0.4186
Epoch 5/50
6/6 - 1s - loss: 1.6266 - accuracy: 0.4097 - val_loss: 1.6407 - val_accuracy: 0.4264
Epoch 6/50
6/6 - 1s - loss: 1.5889 - accuracy: 0.4184 - val_loss: 1.5773 - val_accuracy: 0.4341
Epoch 7/50
6/6 - 1s - loss: 1.5310 - accuracy: 0.4437 - val_loss: 1.5227 - val_accuracy: 0.4535
Epoch 8/50
6/6 - 1s - loss: 1.4547 - accuracy: 0.4854 - val_loss: 1.4320 - val_accuracy: 0.4767
Epoch 9/50
6/6 - 1s - loss: 1.3675 - accuracy: 0.5252 - val_loss: 1.3652 - val_accuracy: 0.5116
Epoch 10/50
6/6 - 1s - loss: 1.2913 - accuracy: 0.5398 - val_loss: 1.2512 - val_accuracy: 0.5659
Epoch 11/50
6/6 - 1s - loss: 1.1730 - a

In [8]:
np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 1s - loss: 1.9786 - accuracy: 0.2379 - val_loss: 1.8545 - val_accuracy: 0.1628
Epoch 2/50
6/6 - 1s - loss: 1.7705 - accuracy: 0.3398 - val_loss: 1.7468 - val_accuracy: 0.4186
Epoch 3/50
6/6 - 1s - loss: 1.7135 - accuracy: 0.4097 - val_loss: 1.6935 - val_accuracy: 0.4186
Epoch 4/50
6/6 - 1s - loss: 1.6765 - accuracy: 0.4097 - val_loss: 1.6843 - val_accuracy: 0.4186
Epoch 5/50
6/6 - 1s - loss: 1.6665 - accuracy: 0.4097 - val_loss: 1.6877 - val_accuracy: 0.4186
Epoch 6/50
6/6 - 1s - loss: 1.6601 - accuracy: 0.4097 - val_loss: 1.6790 - val_accuracy: 0.4186
Epoch 7/50
6/6 - 1s - loss: 1.6604 - accuracy: 0.4097 - val_loss: 1.6769 - val_accuracy: 0.4186
Epoch 8/50
6/6 - 1s - loss: 1.6511 - accuracy: 0.4097 - val_loss: 1.6788 - val_accuracy: 0.4186
Epoch 9/50
6/6 - 1s - loss: 1.6503 - accuracy: 0.4097 - val_loss: 1.6784 - val_accuracy: 0.4186
Epoch 10/50
6/6 - 1s - loss: 1.6478 - accuracy: 0.4097 - val_loss: 1.6609 - val_accuracy: 0.4186
Epoch 11/50
6/6 - 1s - loss: 1.6319 - a

In [9]:
np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (10, 10), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 1s - loss: 1.9773 - accuracy: 0.2602 - val_loss: 1.8231 - val_accuracy: 0.1628
Epoch 2/50
6/6 - 1s - loss: 1.7422 - accuracy: 0.3583 - val_loss: 1.6999 - val_accuracy: 0.4186
Epoch 3/50
6/6 - 1s - loss: 1.6789 - accuracy: 0.4097 - val_loss: 1.6853 - val_accuracy: 0.4186
Epoch 4/50
6/6 - 1s - loss: 1.6629 - accuracy: 0.4097 - val_loss: 1.6951 - val_accuracy: 0.4186
Epoch 5/50
6/6 - 1s - loss: 1.6580 - accuracy: 0.4097 - val_loss: 1.7191 - val_accuracy: 0.4186
Epoch 6/50
6/6 - 1s - loss: 1.6641 - accuracy: 0.4097 - val_loss: 1.6742 - val_accuracy: 0.4186
Epoch 7/50
6/6 - 1s - loss: 1.6402 - accuracy: 0.4097 - val_loss: 1.6647 - val_accuracy: 0.4186
Epoch 8/50
6/6 - 1s - loss: 1.6054 - accuracy: 0.4107 - val_loss: 1.6089 - val_accuracy: 0.4186
Epoch 9/50
6/6 - 1s - loss: 1.5783 - accuracy: 0.4126 - val_loss: 1.6148 - val_accuracy: 0.4884
Epoch 10/50
6/6 - 1s - loss: 1.5384 - accuracy: 0.4699 - val_loss: 1.5422 - val_accuracy: 0.4302
Epoch 11/50
6/6 - 1s - loss: 1.4753 - a

## Task 4

1. Try different word count in the next-word prediction task with a)8 and b)12 previous words and report accuracy for each. Use the same Sherlock data.

2. Try different LSTM architectures and comment on model performace for each. 
    - Add a dropout layer with 0.25
    - Change number of neurons to a) 64 b) 256 neurons
    
    Train for 10 epochs 

In [16]:
path = 'sherlock.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 581889


In [17]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [18]:
# unique sorted words list
unique_words = np.unique(words)

# dictionary(<key: value>) with each word form the unique_words list as key and its corresponding position as value.
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [19]:
WORD_LENGTH = 8
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print('sequence:', prev_words[10101])
print('next word in sequence:', next_words[10101])

sequence: ['of', 'twenty', 'one', 'years', 'are', 'eligible', 'apply', 'in']
next word in sequence: person


In [20]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [21]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

np.random.seed(2002)
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

Epoch 1/2
811/811 [==============================] - 195s 240ms/step - loss: 6.0031 - accuracy: 0.1082 - val_loss: 7.0025 - val_accuracy: 0.1047
Epoch 2/2
811/811 [==============================] - 198s 244ms/step - loss: 5.7841 - accuracy: 0.1480 - val_loss: 7.7806 - val_accuracy: 0.1099


In [22]:
model = Sequential()
model.add(LSTM(256, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Dropout(0.25))
model.add(Activation('softmax'))

np.random.seed(2002)
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

Epoch 1/2
811/811 [==============================] - 388s 478ms/step - loss: 7.1230 - accuracy: 0.0864 - val_loss: 7.2727 - val_accuracy: 0.0837
Epoch 2/2
811/811 [==============================] - 389s 479ms/step - loss: 7.1390 - accuracy: 0.1190 - val_loss: 7.8540 - val_accuracy: 0.0923
